In [ ]:
%load_ext autoreload

In [ ]:
%autoreload

In [1]:
import os
import json
from collections import defaultdict
from pprint import pprint
from feature_extractors import NDVIFeatureExtractor, ColorChannelsFeatureExtractor, LBPFeatureExtractor
from pipeline import complete_pipeline
from sklearn.pipeline import FeatureUnion

In [2]:
import sys
sys.path.append("../")
from config import RESULTS_PATH

# Initialise feature extractors

In [12]:
feature_extractors = [
    ('spectral', FeatureUnion([
        ("channels", ColorChannelsFeatureExtractor()),
        ("ndvi", NDVIFeatureExtractor())
    ]))
]

In [13]:
textural_features = [
    ('lbp_1_8', LBPFeatureExtractor(r=1)),
    ('lbp_2_16', LBPFeatureExtractor(r=2)),
    ('lbp_3_24', LBPFeatureExtractor(r=3))
]
feature_extractors.extend(textural_features)
feature_extractors.append(('lbp_multiscale', FeatureUnion(textural_features)))

In [14]:
feature_extractors

[('spectral', FeatureUnion(n_jobs=None,
         transformer_list=[('channels', <feature_extractors.ColorChannelsFeatureExtractor object at 0x1293134a8>), ('ndvi', <feature_extractors.NDVIFeatureExtractor object at 0x129313518>)],
         transformer_weights=None)),
 ('lbp_1_8', <feature_extractors.LBPFeatureExtractor at 0x129dbb470>),
 ('lbp_2_16', <feature_extractors.LBPFeatureExtractor at 0x129dbb4e0>),
 ('lbp_3_24', <feature_extractors.LBPFeatureExtractor at 0x129dbb550>),
 ('lbp_multiscale', FeatureUnion(n_jobs=None,
         transformer_list=[('lbp_1_8', <feature_extractors.LBPFeatureExtractor object at 0x129dbb470>), ('lbp_2_16', <feature_extractors.LBPFeatureExtractor object at 0x129dbb4e0>), ('lbp_3_24', <feature_extractors.LBPFeatureExtractor object at 0x129dbb550>)],
         transformer_weights=None))]

# evaluate performance

In [15]:
performances = defaultdict(dict)
for feature_name, feature_extractor in feature_extractors:
    performance = complete_pipeline(feature_extractor)
    performances[feature_name] = performance

/Users/Samu/anaconda/envs/vision/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/Samu/anaconda/envs/vision/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Samu/anaconda/envs/vision/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/Samu/anaconda/envs/vision/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average

In [5]:
performances

defaultdict(dict,
            {'ndvi': {'mean_f2': 0.7074426461162135,
              'per_class_f2': OrderedDict([('primary', 0.977069321612032),
                           ('clear', 0.8457932733431416),
                           ('agriculture', 0.49740629183400276),
                           ('road', 0.27055702917771884),
                           ('water', 0.47584921886197507),
                           ('partly_cloudy', 0.1408783252006926),
                           ('cultivation', 0.07210919392222509),
                           ('habitation', 0.06301875192130342),
                           ('haze', 0.36451748841129367),
                           ('cloudy', 0.5005268703898841),
                           ('bare_ground', 0.01360544217687075),
                           ('selective_logging', 0.0),
                           ('artisinal_mine', 0.0),
                           ('blooming', 0.0),
                           ('slash_burn', 0.0),
                           ('convent

In [6]:
for feature_name, performance in performances.items():
    file_path = os.path.join(RESULTS_PATH, f'{feature_name}.json')
    with open(file_path, 'w') as f:
        json.dump(performance, f, indent=4)